In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
a = np.array([1.26667, 1.63, 1.8, 1.083])
a = a.reshape(1,-1)
b = np.array([0.75, 3.4, 0.9,2.7])
b = b.reshape(1, -1)

ans = cosine_similarity(a, b)
print(ans)

[[0.83326385]]


# Project

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from math import sqrt  
import os
import io
import random

from sklearn import model_selection
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score
import xgboost
from sklearn.tree import DecisionTreeRegressor
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

In [57]:
true_df = pd.read_csv('data/fake-news/True.csv')
fake_df = pd.read_csv('data/fake-news/Fake.csv')

## EDA of True.csv

In [50]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
true_df.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [6]:
true_df.dtypes

title      object
text       object
subject    object
date       object
dtype: object

In [7]:
true_df['subject'].value_counts()

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [40]:
true_df['text'][random.randint(0,10000)]

'The following statements\xa0were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own.\xa0Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - The Massive Tax Cuts, which the Fake News Media is desperate to write badly about so as to please their Democrat bosses, will soon be kicking in and will speak for themselves. Companies are already making big payments to workers. Dems want to raise taxes, hate these big Cuts! [0724 EST] - Was @foxandfriends just named the most influential show in news? You deserve it - three great people! The many Fake News Hate Shows should study your formula for success! [0745 EST] - Home Sales hit BEST numbers in 10 years! MAKE AMERICA GREAT AGAIN [0856 EST] - House Democrats want a SHUTDOWN for the holidays in order to distract from the very popular, just passed, Tax Cuts. House Republicans, don’t let this happen. Pass the C.R. TODAY and k

In [41]:
total_len = 11272
total = 0
for i in range(total_len+1):
    total += len((true_df['text'][i]).split())

print(total/total_len)

408.1515259048971


In [44]:
true_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [58]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [45]:
fake_df.isnull().sum()

title               0
text                0
subject            21
date               21
Unnamed: 4      23481
                ...  
Unnamed: 167    23500
Unnamed: 168    23500
Unnamed: 169    23500
Unnamed: 170    23500
Unnamed: 171    23500
Length: 172, dtype: int64

In [43]:
fake_df.describe()

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171
count,23502,23502,23481,23481,21,20,20,20,18,18,...,2,2,2,2,2,2,2,2,2,2
unique,17914,17466,17,1692,11,10,11,10,9,9,...,1,1,1,1,1,1,1,1,2,1
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017",but the newly circulated memo states that Tip...,Stone enlisted some 200 hardened criminals fo...,in the tradition of the former versions of Wo...,the strategy works splendidly for a while,to lurking spymasters,but he did use the power of his office to fi...,...,is that it would be very easy for an agent pr...,and it s certain that the powers that be will...,it s important to separate the fallout from t...,as those in mainstream media such as The New ...,The Washington Post and the self-anointed fa...,seem to have propped up this pizzeria conspir...,but for the moment,it seems to have collapsed under the weight o...,US_News,"December 28, 2016"
freq,6,626,9050,46,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,1,2


In [15]:
fake_df['subject'].value_counts()

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64